<a href="https://colab.research.google.com/github/Slazenzer/Multi-Agent-System/blob/main/Multi_Agent_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q crewai crewai-tools langchain-groq langchain-google-genai google-generativeai pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 665.2/665.2 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 kB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 8.8 MB/s 

In [ ]:
import os
import nest_asyncio
from google.colab import userdata
from getpass import getpass

nest_asyncio.apply()
print("Please enter your API Keys (press Enter after pasting each):")

os.environ["GOOGLE_API_KEY"] = getpass("Enter Google Gemini API Key: ")
os.environ["GROQ_API_KEY"] = getpass("Enter Groq API Key: ")
os.environ["TAVILY_API_KEY"] = getpass("Enter Tavily API Key: ")

print("\nKeys successfully set!")

Please enter your API Keys (press Enter after pasting each):
Enter Google Gemini API Key: ··········
Enter Groq API Key: ··········
Enter Tavily API Key: ··········

Keys successfully set!


In [ ]:

from crewai.tools import BaseTool
import google.generativeai as genai
from PIL import Image
import os

class ImageToCodeTool(BaseTool):
    name: str = "Image Code Extractor"
    description: str = "Takes an image file path, reads the image, and extracts code and error messages."

    def _run(self, file_path: str) -> str:
        try:
            # Configure Gemini
            genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
            model = genai.GenerativeModel('gemini-pro')

            # Load Image
            img = Image.open(file_path)

            # Prompt the Vision Model
            prompt = "Extract the code snippet and the exact error message present in this image. Return only the code and the error text."
            response = model.generate_content([prompt, img])

            return response.text
        except Exception as e:
            return f"Error processing image: {str(e)}"

print("Custom Vision Tool created successfully.")

Custom Vision Tool created successfully.


In [ ]:
!pip install tavily-python

In [ ]:
!pip install litellm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 155.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 80.8 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.76.0
    Uninstalling grpcio-1.76.0:
      Successfully uninstalled grpcio-1.76.0
  Attempting uninstall: openai
    Found existing installation: openai 1.83.0
    Uninstalling openai-1.83.0:
      Successfully uninstalled openai-1.83.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
crewai 1.7.0 requires openai~=1.83.0, but you have openai 2.9.0 which is incompatible.
instructor 1.12.0 requires openai<2.0.0,>=1.70.0, but you have openai 2.9.0 which is incompatible.
google-adk 1.

In [ ]:

import os
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import BaseTool
from tavily import TavilyClient

class CustomSearchTool(BaseTool):
    name: str = "Internet Search Tool"
    description: str = "Search the web for technical solutions using Tavily."

    def _run(self, query: str) -> str:
        try:
            tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
            response = tavily_client.search(query=query)
            results = []
            if 'results' in response:
                for res in response['results'][:3]:
                    results.append(f"Source: {res['title']}\nContent: {res['content']}\n")
            return "\n".join(results)
        except Exception as e:
            return f"Error performing search: {str(e)}"

groq_llm = LLM(
    model="groq/llama-3.3-70b-versatile",
    api_key=os.environ["GROQ_API_KEY"]
)

# 3. Initialize Tools
vision_tool = ImageToCodeTool()
search_tool = CustomSearchTool()

# 4. Define Agents

extractor = Agent(
    role='Code Extractor',
    goal='Extract code and error text from image files.',
    backstory='You are an optical character recognition expert specialized in programming screenshots.',
    tools=[vision_tool],
    llm=groq_llm,
    verbose=True
)

researcher = Agent(
    role='StackOverflow Researcher',
    goal='Find the correct solution for the extracted error.',
    backstory='You are an expert at searching technical documentation and forums to find bug fixes.',
    tools=[search_tool],
    llm=groq_llm,
    verbose=True
)

fixer = Agent(
    role='Senior Developer',
    goal='Write the corrected code and explain the fix.',
    backstory='You provide safe, clean, and commented code fixes based on research.',
    llm=groq_llm,
    verbose=True
)

# 5. Define Tasks

task_extract = Task(
    description='Use the "Image Code Extractor" tool to analyze the image at path: "{image_path}". Extract the code and the error message.',
    expected_output='Text containing the Code Snippet and Error Message.',
    agent=extractor
)

task_search = Task(
    description='Take the extracted error and code from the previous task. Search the web for a solution to this specific error.',
    expected_output='A summary of the solution found.',
    agent=researcher
)

task_fix = Task(
    description='Based on the research, provide the CORRECTED code. Explain step-by-step how to apply the fix.',
    expected_output='1. The Corrected Code Block\n2. Explanation of the solution.',
    agent=fixer
)

# 6. Assemble Crew
crew = Crew(
    agents=[extractor, researcher, fixer],
    tasks=[task_extract, task_search, task_fix],
    process=Process.sequential,
    verbose=True
)

print("Crew assembled successfully with Groq Native LLM.")

Crew assembled successfully with Groq Native LLM.


In [ ]:
from google.colab import files

print("Please upload your screenshot containing the code/error:")
uploaded = files.upload()

if uploaded:
    image_filename = list(uploaded.keys())[0]
    print(f"\nProcessing image: {image_filename}...\n")

    result = crew.kickoff(inputs={'image_path': image_filename})

    from IPython.display import Markdown
    display(Markdown("## 🛠️ **Final Code Fix**"))
    display(Markdown(str(result)))
else:
    print("No file uploaded.")

Please upload your screenshot containing the code/error:


Saving Screenshot 2025-12-12 003204.png to Screenshot 2025-12-12 003204 (2).png

Processing image: Screenshot 2025-12-12 003204 (2).png...



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 44457403-674a-4cc7-af77-7e02c2fb96e1                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Code Extractor                                                                                          │
│                                                                                                                 │
│  Task: Use the "Image Code Extractor" tool to analyze the image at path: "Screenshot 2025-12-12 003204          │
│  (2).png". Extract the code and the error message.                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: StackOverflow Researcher                                                                                │
│                                                                                                                 │
│  Thought: Thought: The error message indicates that the API endpoint used by the tool is returning a 404        │
│  error, which means that the model "gemini-1.5-flash" is not found for the API version "v1beta" or is not       │
│  supported for the "generateContent" method. To solve this issue, I need to search for a solution on the web    │
│  that can provide more information about the available models and their supported methods for the Google        │
│  Generative Language API.                                                                                       │
│                                                                                                                 │
│  Using Tool: Internet Search Tool                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: StackOverflow Researcher                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The solution to the error "Error processing image: 404 POST                                                    │
│  https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-e  │
│  ncoding%3Dint: models/gemini-1.5-flash is not found for API version v1beta, or is not supported for            │
│  generateContent. Call ListModels to see the list of available models and their supported methods" is to use    │
│  the ListModels method to retrieve the list of available models and their supported methods for the Google      │
│  Generative Language API.                                                                                       │
│                                                                                                                 │
│  According to the Google Generative Language API documentation, the ListModels method can be used to            │
│  programmatically list the available models and retrieve extended metadata such as supported functionality and  │
│  other information.                                                                                             │
│                                                                                                                 │
│  The available models for the Google Generative Language API can be found on the [models                        │
│  page](https://ai.google.dev/gemini-api/docs/models).                                                           │
│                                                                                                                 │
│  To use the Gemini model with the AI SDK, you can use the following optional provider options:                  │
│                                                                                                                 │
│  *   Only supported by specific Google Generative AI models.                                                    │
│  *   For more information about the possible value ranges for each model, see the [Google Generative AI         │
│  thinking documentation](https://ai.google.dev/gemini-api/docs/thinking).                                       │
│  *   Only supported by specific Google Generative AI models.                                                    │
│  *   See the [models page](https://ai.google.dev/gemini-api/docs/models) to check if caching is supported for   │
│  the used model.                                                                                                │
│                                                                                                                 │
│  You can use the following code to generate content with a given model:                                         │
│                                                                                                                 │
│  ```javascript                                                                                                  │
│  generateContent({                                                                                              │
│      model: 'gemini-2.0-flash',                                                                                 │
│      contents: 'why is the sky blue?',                                                                          │
│      config: {                                         

Output()

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Developer                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The Corrected Code Block:                                                                                      │
│  ```javascript                                                                                                  │
│  // Define the function to generate content with a given model                                                  │
│  async function generateContent(options) {                                                                      │
│      // Set the API endpoint URL                                                                                │
│      const url = 'https://generativelanguage.googleapis.com/v1beta/models/' + options.model +                   │
│  ':generateContent';                                                                                            │
│                                                                                                                 │
│      // Set the API request headers                                                                             │
│      const headers = {                                                                                          │
│          'Content-Type': 'application/json',                                                                    │
│      };                                                                                                         │
│                                                                                                                 │
│      // Set the API request body                                                                                │
│      const body = {                                                                                             │
│          contents: options.contents,                                                                            │
│          config: options.config,                                                                                │
│      };                                                                                                         │
│                                                                                                                 │
│      // Send the API request                                                                                    │
│      const response = await fetch(url, {                                                                        │
│          method: 'POST',                                                                                        │
│          headers: headers,                                                                                      │
│          body: JSON.stringify(body),                                                                            │
│      });                                                                                                        │
│                                                                                                                 │
│      // Check if the response was successful                                                                    │
│      if (response.ok) {                                                                                         │
│          // Return the response data                                                                            │
│          return await response.json();                 